<a href="https://colab.research.google.com/github/NIHITHPENUMUCHU/person-detection-tracking/blob/main/person_detection_and_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Install required libraries
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install deep_sort

ERROR: Could not find a version that satisfies the requirement deep_sort (from versions: none)
ERROR: No matching distribution found for deep_sort


In [10]:
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/ZQPei/deep_sort_pytorch.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16959, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 16959 (delta 78), reused 108 (delta 57), pack-reused 16805 (from 1)
Receiving objects: 100% (16959/16959), 15.70 MiB | 16.13 MiB/s, done.
Resolving deltas: 100% (11616/11616), done.
Cloning into 'deep_sort_pytorch'...
remote: Enumerating objects: 1078, done.
remote: Counting objects: 100% (352/352), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 1078 (delta 155), reused 261 (delta 90), pack-reused 726 (from 1)
Receiving objects: 100% (1078/1078), 29.15 MiB | 17.11 MiB/s, done.
Resolving deltas: 100% (506/506), done.


In [11]:
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch.git

fatal: destination path 'yolov5' already exists and is not an empty directory.
Cloning into 'Yolov5_DeepSort_Pytorch'...
remote: Enumerating objects: 13782, done.
remote: Counting objects: 100% (2521/2521), done.
remote: Compressing objects: 100% (916/916), done.
remote: Total 13782 (delta 1774), reused 2235 (delta 1605), pack-reused 11261 (from 1)
Receiving objects: 100% (13782/13782), 111.73 MiB | 27.14 MiB/s, done.
Resolving deltas: 100% (8724/8724), done.


In [12]:
!pip install -qr yolov5/requirements.txt
!pip install -q torch torchvision torchaudio

In [18]:
!wget https://drive.google.com/drive/folders/1KCa19B4kpZwhTUTtGyuvR7h_w47OFBAm?usp=sharing -O test_video.mp4

--2024-09-05 18:43:50--  https://drive.google.com/drive/folders/1KCa19B4kpZwhTUTtGyuvR7h_w47OFBAm?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.139.139, 74.125.139.138, 74.125.139.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.139.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘test_video.mp4’

test_video.mp4          [  <=>               ] 265.65K  1.24MB/s    in 0.2s    

2024-09-05 18:43:51 (1.24 MB/s) - ‘test_video.mp4’ saved [272021]



In [25]:
!pip install -e ./Yolov5_DeepSort_Pytorch


Obtaining file:///content/Yolov5_DeepSort_Pytorch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for boxmot (pyproject.toml) ... done
  Created wheel for boxmot: filename=boxmot-10.0.80-py3-none-any.whl size=18845 sha256=9e9d588702c19fe16a061fc3ea9541b1426b666177ab9f68af895bc4da6cc164
  Stored in directory: /tmp/pip-ephem-wheel-cache-s_6pgxln/wheels/fc/bc/40/f486ad989dde83e8e9523342a9e186d4d30314756dc042510b
Successfully built boxmot
  Attempting uninstall: boxmot
    Found existing installation: boxmot 10.0.80
    Uninstalling boxmot-10.0.80:
      Successfully uninstalled boxmot-10.0.80


In [27]:
import sys
from pathlib import Path

yolov5_path = Path("yolov5")
deep_sort_path = Path("Yolov5_DeepSort_Pytorch") / 'deep_sort'  # Ensure deep_sort is correctly referenced

sys.path.append(str(yolov5_path))
sys.path.append(str(deep_sort_path))


In [28]:
!ls Yolov5_DeepSort_Pytorch


assets	CITATION.cff  LICENSE	   poetry.lock	   README.md	  tests
boxmot	Dockerfile    MANIFEST.in  pyproject.toml  renovate.json  tracking


In [32]:
import sys
import cv2
import torch
import numpy as np
from pathlib import Path

# Add YOLOv5 and DeepSort directories to path
yolov5_path = Path("yolov5")
deep_sort_path = Path("Yolov5_DeepSort_Pytorch")
sys.path.append(str(yolov5_path))
sys.path.append(str(deep_sort_path))

from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_boxes
from yolov5.utils.torch_utils import select_device
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker

# Load YOLOv5 model
device = select_device('')
model = attempt_load('yolov5s.pt', device=device)

# Initialize Deep SORT
max_cosine_distance = 0.3
nn_budget = None
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

def detect_and_track(frame, model, tracker):
    # Prepare image
    img = torch.from_numpy(frame).to(device).float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img)[0]

    # Apply NMS
    pred = non_max_suppression(pred, 0.4, 0.5)

    # Process detections
    detections = []
    if pred[0] is not None:
        for *xyxy, conf, cls in pred[0]:
            if int(cls) == 0:  # Only track persons (class 0)
                bbox = torch.tensor(xyxy).view(1, 4).view(-1).tolist()
                detections.append(Detection(bbox, conf))

    # Update tracker
    tracker.predict()
    tracker.update(detections)

    # Draw results
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        bbox = track.to_tlbr()
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255,0,0), 2)
        cv2.putText(frame, f"ID: {track.track_id}", (int(bbox[0]), int(bbox[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)

    return frame

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_and_track(frame, model, tracker)
        out.write(frame)

    cap.release()
    out.release()

# Process the video
input_path = 'test_video.mp4'
output_path = 'output_video.mp4'
process_video(input_path, output_path)

ModuleNotFoundError: No module named 'deep_sort'